## Creating the ES module
We will first create a single self contained ES module, since using threejs-fiber from esm.sh does not work flawlessly.

First create a simple file called `threejs-fiber.js` that exports what we need.

```javascript
export * from '@react-three/fiber'
export * from "@react-three/drei";
```

Next, we install the libraries:

```bash
$ npm install @react-three/drei@9.68.6 @react-three/fiber@8.13.0
```

And use ESBuild to turn this into a self-contained module/bundle, without react, since ipyreact provides that for us.

```
$ npx esbuild ./threejs-fiber.js --bundle --outfile=./threejs-fiber.bundle.js --format=esm --external:react --external:react-dom --external:react-reconciler --external:react-reconciler/constants --target=esnext
```


Now we can define the module with a custom name (we call it threejs-fiber).

In [ ]:
%pip install -q ipyreact
# This line is needed for JupyterLite

In [ ]:
import ipyreact
from pathlib import Path
ipyreact.define_module("threejs-fiber", Path("./threejs-fiber.bundle.js"))

In [ ]:
from traitlets import default


class BoxWidget(ipyreact.Widget):
    _esm = """
        import React, { useRef, useState } from "react"
        import { Canvas, useFrame, useThree } from 'threejs-fiber'
        import { OrbitControls } from "threejs-fiber";

        export default function Box({position, color}) {
          const ref = useRef()
          useFrame(() => (ref.current.rotation.x = ref.current.rotation.y += 0.01))

          return (
            <mesh position={position} ref={ref}>
              <boxGeometry args={[1, 1, 1]} attach="geometry" />
              <meshPhongMaterial color={color} attach="material" />
            </mesh>
          )
        }

    """

In [ ]:
import random


def random_color():
    # Generates a random hex color code
    return "#" + ''.join([random.choice('0123456789ABCDEF') for _ in range(6)])

def add(_ignore=None):
    x = random.random() * 4 - 2
    z = random.random() * 4 - 1
    color = random_color()  # Call the random_color function to get a random color
    box = BoxWidget(props=dict(position=[x, 0, z], color=color))  # Use the random color for the box
    canvas.children = [*canvas.children, box]

canvas = ipyreact.Widget(_type="Canvas", _module="threejs-fiber",
        events=dict(onClick=add),
        children=[
            BoxWidget(props=dict(position=[-1, 0, 3], color="#18a36e")),
            BoxWidget(props=dict(position=[1, 0, 3], color="#f56f42")),
            ipyreact.Widget(_type="OrbitControls", _module="threejs-fiber"),
            # seems that if it starts with a small letter, it's globally available, and not exported
            # from the threejs-fiber module, therefore we do not pass _module="threejs-fiber"
            ipyreact.Widget(_type="directionalLight",
                           props=dict(color="#ffffff", intensity=1, position=[-1, 2, 4]))
       ]
)

# the canvas fills the parent, so wrap it in a div with the fixed height
ipyreact.Widget(_type="div", props=dict(style=dict(height="600px")), children=[canvas])

# Using in solara

Although this shows the power of ipyreact, in how it composes, we can do better.

The first problem is that it does not have a very nice API, it's very low level.

The second problem is that although we now have a nice composable set of widgets, actually building a larger application by manually adding and removing widgets is tiresome, which is why we build [Solara](https://solara.dev).

By following [the solara docs on how to use widgets](https://solara.dev/docs/howto/ipywidget-libraries) we can add wrapper component with a nicer API.

In [ ]:
import solara

@solara.component
def Box(position, color, props={}, events={}, children=[]):
    return BoxWidget.element(props={**props, **dict(color=color, position=position)}, events=events, children=children)


@solara.component
def Canvas(props={}, events={}, children=[]):
    return ipyreact.Widget.element(props=props, events=events, children=children, _type="Canvas", _module="threejs-fiber")


@solara.component
def OrbitControls(props={}, events={}, children=[]):
    return ipyreact.Widget.element(_type="OrbitControls", _module="threejs-fiber", props=props, events=events, children=children)


@solara.component
def DirectionalLight(props={}, events={}, children=[]):
    # starts with a lower case, should be available globally, so we don't need to pass
    # _module="threejs-fiber"
    return ipyreact.Widget.element(_type="directionalLight", props=props, events=events, children=children)


@solara.component
def Div(style={}, props={}, events={}, children=[]):
    # we use a ipyreact based div to avoid an extra wrapper div which will affect layout
    return ipyreact.Widget.element(_type="div", props={**props, **dict(style=style)}, children=children, events=events)

Now we can build a dynamic application without having to worry about how to add and remove widgets, and populating our scene dynamically based on data (the reactive variable). On top of that we also have a nicer API that we can customize to our needs.

In [ ]:
boxes = solara.reactive([
    ([-1, 0, 3], "#18a36e"),
    ([1, 0, 3], "#f56f42"),
])
    
def add(event_data=None):
    x = random.random() * 4 - 2
    z = random.random() * 4 - 1
    color = random_color()  # Call the random_color function to get a random color
    boxes.value = [*boxes.value, ([x, 0, z], color)]


def clear():
    boxes.value = boxes.value[:2]


def add_10():
    for i in range(10):
        add()
        
@solara.component
def Page():
    with solara.Row():
        solara.Button("Clear", on_click=clear)
        solara.Button("Add 10", on_click=add_10)
    solara.Markdown("Click to add a new box")
    with Div(style={"height": "600px"}):
        # a canvas fill the available space, so we add a parent div with height
        with Canvas(events={"onClick": add}):
            for position, color in boxes.value:
                Box(position=position, color=color)
            OrbitControls()
            DirectionalLight(props=dict(color="#ffffff", intensity=1, position=[-1, 2, 4]))
Page()